# Vox Amelior - Full Stack Colab Setup

This notebook starts **ALL** core services on Google Colab.

**Prerequisites:**
1. Set `NGROK_AUTHTOKEN` in Colab Secrets
2. Set `SESSION_KEY` and `USERS_DB_KEY` (random strings)
3. (Optional) Set `HF_TOKEN` for Gemma models

**Services Started:**
- API Gateway (port 8000)
- Gemma Service (port 8001)
- Transcription Service (port 8003)
- RAG Service (port 8004)
- Emotion Service (port 8005)
- ML Service (port 8006)
- Insights Service (port 8010)
- Fiserv Service (port 8008)

In [ ]:
# @title 1. Mount Drive & Clone Repository
import os
import sys
from google.colab import drive

drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Nemo_Server"
REPO_PATH = os.path.join(DRIVE_BASE, "repo")
LIB_CACHE = os.path.join(DRIVE_BASE, "lib_cache")

os.makedirs(LIB_CACHE, exist_ok=True)
os.makedirs(REPO_PATH, exist_ok=True)

if not os.path.exists(os.path.join(REPO_PATH, "services")):
    print("Cloning repository...")
    !git clone https://github.com/pruittcolon/Nemo_Server.git "$REPO_PATH"
else:
    print("Repository found on Drive.")

%cd $REPO_PATH
print(f"Working directory: {os.getcwd()}")

In [ ]:
# @title 2. Install System Dependencies
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 -qq
# Optional: install libsqlcipher if we were using real encryption, but we'll mock it for Colab
print("System dependencies installed.")

In [ ]:
# @title 3. Install Python Dependencies
import os
import sys

LIB_CACHE = "/content/drive/MyDrive/Nemo_Server/lib_cache"
if LIB_CACHE not in sys.path:
    sys.path.insert(0, LIB_CACHE)

cache_files = os.listdir(LIB_CACHE) if os.path.exists(LIB_CACHE) else []

if len(cache_files) < 15: # Bumped check count
    print("Installing dependencies to Drive cache...")
    !pip install -t "$LIB_CACHE" \
        fastapi uvicorn pydantic httpx pyngrok nest_asyncio \
        python-multipart python-jose bcrypt cryptography \
        soundfile librosa ffmpeg-python redis faiss-cpu \
        sentence-transformers llama-cpp-python transformers \
        pandas numpy scikit-learn scipy \
        --quiet
    print("Dependencies cached.")
else:
    print(f"Cache found ({len(cache_files)} items).")

In [ ]:
# @title 4. Configure Environment & Monkeypatch
import os
import sys
from google.colab import userdata

# 1. Load secrets
try:
    os.environ["NGROK_AUTHTOKEN"] = userdata.get('NGROK_AUTHTOKEN')
    os.environ["SESSION_KEY"] = userdata.get('SESSION_KEY')
    os.environ["USERS_DB_KEY"] = userdata.get('USERS_DB_KEY')
    try: os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    except: pass
except Exception as e:
    print(f"Warning loading secrets: {e}")

# 2. Service URLs (localhost mode)
os.environ["GEMMA_URL"] = "http://localhost:8001"
os.environ["TRANSCRIPTION_URL"] = "http://localhost:8003"
os.environ["RAG_URL"] = "http://localhost:8004"
os.environ["EMOTION_URL"] = "http://localhost:8005"
os.environ["ML_SERVICE_URL"] = "http://localhost:8006"
os.environ["INSIGHTS_URL"] = "http://localhost:8010"
os.environ["FISERV_URL"] = "http://localhost:8008"

# 3. Paths & Config
DATA_DIR = "/content/drive/MyDrive/Nemo_Server/data"
os.environ["DB_PATH"] = f"{DATA_DIR}/rag.db"
os.environ["FAISS_INDEX_PATH"] = f"{DATA_DIR}/faiss_index"
os.environ["APP_INSTANCE_DIR"] = f"{DATA_DIR}/instance"
os.environ["TEST_MODE"] = "true"
os.environ["JWT_ONLY"] = "false"

os.makedirs(f"{DATA_DIR}/faiss_index", exist_ok=True)
os.makedirs(f"{DATA_DIR}/instance", exist_ok=True)

# 4. Monkeypatch pysqlcipher3 for Insights Service
# Insights service wants encryption, but installing pysqlcipher3 on Colab is a pain.
# We mock it with standard sqlite3 for this demo.
import sqlite3
sys.modules['pysqlcipher3'] = sqlite3
sys.modules['pysqlcipher3.dbapi2'] = sqlite3

print("Environment configured & patched.")

In [ ]:
# @title 5. Start ALL Services
import threading
import time
import sys
import os
import uvicorn
import nest_asyncio

nest_asyncio.apply()

# Update path for imports
REPO_PATH = "/content/drive/MyDrive/Nemo_Server/repo"
sys.path.insert(0, REPO_PATH)
for service in ["api-gateway", "gemma-service", "transcription-service", 
                "rag-service", "emotion-service", "ml-service", 
                "insights-service", "fiserv-service"]:
    sys.path.insert(0, os.path.join(REPO_PATH, f"services/{service}"))

# Generic launcher
def start_service(name, import_path, port):
    print(f"Starting {name} on port {port}...")
    try:
        uvicorn.run(import_path, host="0.0.0.0", port=port, log_level="error")
    except Exception as e:
        print(f"❌ {name} error: {e}")

# Thread launcher
services = [
    ("API Gateway", "services.api-gateway.src.main:app", 8000),
    ("Gemma", "services.gemma-service.src.main:app", 8001),
    ("Transcription", "services.transcription-service.src.main:app", 8003),
    ("RAG", "services.rag-service.src.main:app", 8004),
    ("Emotion", "services.emotion-service.src.main:app", 8005),
    ("ML", "services.ml-service.src.main:app", 8006),
    ("Fiserv", "services.fiserv-service.src.main:app", 8008),
    ("Insights", "services.insights-service.src.main:app", 8010),
]

threads = []
for name, path, port in services:
    t = threading.Thread(target=start_service, args=(name, path, port), daemon=True)
    t.start()
    time.sleep(2) # Stagger to be nice to CPU

print("\nAll services requested. Waiting 15s for initialization...")
time.sleep(15)
print("Services should be ready.")

In [ ]:
# @title 6. Expose (Choose One)
print("Choose how to access your server:")

# OPTION A: Ngrok (Simplest)
import os
from pyngrok import ngrok

try:
    ngrok.set_auth_token(os.environ["NGROK_AUTHTOKEN"])
    public_url = ngrok.connect(8000).public_url
    print(f"\n🌍 NGROK URL: {public_url}/ui/login.html")
except Exception as e:
    print(f"Ngrok error: {e}")

# OPTION B: Tailscale (Secure / Private)
# Uncomment below if you want to use Tailscale instead
# !curl -fsSL https://tailscale.com/install.sh | sh
# !tailscale up
# print("Tailscale started. Access via local IP.")

In [ ]:
# @title 7. Health Check Check
import requests
print("Check Services:")
ports = [8000, 8001, 8003, 8004, 8005, 8006, 8008, 8010]
for p in ports:
    try:
        r = requests.get(f"http://localhost:{p}/health", timeout=2)
        print(f"  Port {p}: {r.status_code}")
    except:
        print(f"  Port {p}: Down")